In [ ]:
import os

import sagemaker
from sagemaker.session import TrainingInput
from sagemaker.pytorch import PyTorch

In [ ]:
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

role = "arn:aws:iam::357984623133:role/service-role/AmazonSageMaker-ExecutionRole-20210713T114662"

print("Role :", role)
print("Bucket :", bucket)

In [ ]:
app_name = "MySmTest"

#yolov5_version = "6.2"
yolov5_version = "6.1"
#yolov5_version = "6.0"
#yolov5_version = "5.0"

In [ ]:
if yolov5_version >= "6.1":
    hyp_filename = "hyps/hyp.VOC.yaml"
elif yolov5_version == "6.0":
    hyp_filename = "hyps/hyp.finetune.yaml"
elif yolov5_version == "5.0":
    hyp_filename = "hyp.finetune.yaml"
else:
    assert False, f"Unsupported YoloV5 version {yolov5_version}"

In [ ]:
# download
!wget https://github.com/ultralytics/yolov5/archive/refs/tags/v{yolov5_version}.tar.gz -O ./yolov5-v{yolov5_version}.tar.gz
!wget https://github.com/ultralytics/yolov5/releases/download/v{yolov5_version}/yolov5s.pt -O ./yolov5s.pt

# extract source package
!tar xvzf ./yolov5-v{yolov5_version}.tar.gz

In [ ]:
# construct input/output directory structure

os.makedirs( "config", exist_ok=True )

!cp ./yolov5s.pt ./config/
!cp ./yolov5-{yolov5_version}/models/yolov5s.yaml ./config/
!cp ./yolov5-{yolov5_version}/data/{hyp_filename} ./config/hyp.yaml
!cp ./data.yaml ./config/


In [ ]:
!aws s3 sync ./config s3://{bucket}/{app_name}/config

In [ ]:
!cp ./sm_entry_point.py ./yolov5-{yolov5_version}/

In [ ]:
s3_config = f"s3://{bucket}/{app_name}/config"
s3_sources = f"s3://{bucket}/{app_name}/sources"
s3_images = f"s3://{bucket}/{app_name}/images"
s3_labels = f"s3://{bucket}/{app_name}/labels"
s3_results = f"s3://{bucket}/{app_name}/results"

print( "s3_config :", s3_config )
print( "s3_sources :", s3_sources )
print( "s3_images :", s3_images )
print( "s3_labels :", s3_labels )
print( "s3_results :", s3_results )


In [ ]:
estimator = PyTorch(
    entry_point = "sm_entry_point.py",
    source_dir = f"./yolov5-{yolov5_version}",
    py_version="py38",
    framework_version="1.11.0",
    role=role,
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    input_mode='File',
    code_location=s3_sources,
    output_path=s3_results,
    base_job_name=f'{app_name}-yolov5'
)


In [ ]:
inputs = {
    "config": TrainingInput(s3_config),
    "images": TrainingInput(s3_images),
    "labels": TrainingInput(s3_labels),
}

estimator.fit(inputs)

In [ ]:
s3_output = f"s3://{bucket}/{app_name}/results/{estimator._current_job_name}/output/"
print(s3_output)

!aws s3 ls {s3_output}